<a href="https://colab.research.google.com/github/SIDDHI100/Answering_Machine/blob/main/EI(Main).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Emotional Intelligence

##Packages

In [ ]:
!pip install deep_translator ipython requests pandas datetime textblob plotly googlemaps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 20.8 MB/s eta 0:00:00
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=e89e6eb8cde8488d081b4c95db6f004481c6926040dcd7706ac7c2cf549d22d1
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


##Functions

In [ ]:
import requests
import pandas as pd
from datetime import datetime
from deep_translator import GoogleTranslator
from textblob import TextBlob
import plotly.express as px
from IPython.display import display


def get_tweets(username, limit=20):
    url = "https://twitter154.p.rapidapi.com/user/tweets"
    querystring = {
        "username": username,
        "limit": str(limit),
        "include_replies": "false",
        "include_pinned": "false"
    }
    headers = {
        "X-RapidAPI-Key": "f4ad7039cemsha993c9ce7332310p10260bjsn310aa5368d55",
        "X-RapidAPI-Host": "twitter154.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    print(response.status_code)
    if response.status_code == 200:
        print( response.json().get('results', []))
        return response.json().get('results', [])
    else:
        print("Failed to retrieve tweets. Status code:", response.status_code)
        return []

def translate_text(text, source_lang="auto", target_lang="en"):
    translated =GoogleTranslator().translate(text, source=source_lang, target=target_lang)
    return translated

def analyze_sentiment(text):
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

username = "nitin_gadkari"
limit = 20
user_tweets = get_tweets(username, limit)

if user_tweets:
    df = pd.DataFrame(user_tweets)
    df['creation_date'] = pd.to_datetime(df['creation_date'], format='%a %b %d %H:%M:%S %z %Y')
    df['translated_text'] = df['text'].apply(translate_text)
    df['sentiment'] = df['translated_text'].apply(analyze_sentiment)
    df['user_location'] = df['user'].apply(lambda user: user['location'])

    display(df[['creation_date', 'translated_text', 'sentiment','user_location']])

    daily_sentiments = df.groupby([df['creation_date'].dt.date, 'sentiment']).size().unstack(fill_value=0).reset_index()
    melted_df = daily_sentiments.melt(id_vars='creation_date', var_name='sentiment', value_name='count')
    fig = px.line(melted_df, x='creation_date', y='count', color='sentiment',
                  title='Sentiment Distribution Over Time',
                  labels={'creation_date': 'Date', 'count': 'Number of Texts'},
                  line_shape='linear',
                  color_discrete_map={'Positive': 'green', 'Neutral': 'gray', 'Negative': 'red'})
    fig.update_layout(
        legend=dict(title='Sentiment', orientation='h', y=1.1, x=0.5),
        xaxis=dict(tickformat='%Y-%m-%d'),
        yaxis=dict(title='Number of Texts', tickmode='linear'),
        autosize=True,
        template='plotly_dark'
    )
    fig.show()
else:
    print("No tweets found for the user:", username)


In [ ]:
import requests
import pandas as pd
from datetime import datetime
from deep_translator import GoogleTranslator
from textblob import TextBlob
import plotly.express as px
from IPython.display import display

def get_tweets_by_hashtag(hashtag, limit=20):
    url = "https://twitter154.p.rapidapi.com/hashtag/hashtag"
    querystring = {
        "hashtag": hashtag,
        "limit": str(limit),
        "section": "top"
    }
    headers = {
        "X-RapidAPI-Key": "f4ad7039cemsha993c9ce7332310p10260bjsn310aa5368d55",
        "X-RapidAPI-Host": "twitter154.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        hashtag_tweets = response.json().get('results', [])
        return hashtag_tweets
    else:
        print("Failed to retrieve tweets by hashtag. Status code:", response.status_code)
        return []

def translate_text(text, source_lang="auto", target_lang="en"):
    translated = GoogleTranslator().translate(text, source=source_lang, target=target_lang)
    return translated

def analyze_sentiment(text):
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

hashtag = "#UPSC"
limit = 20
hashtag_tweets = get_tweets_by_hashtag(hashtag, limit)

if hashtag_tweets:
    df = pd.DataFrame(hashtag_tweets)
    df['creation_date'] = pd.to_datetime(df['creation_date'], format='%a %b %d %H:%M:%S %z %Y')
    df['translated_text'] = df['text'].apply(translate_text)
    df['sentiment'] = df['translated_text'].apply(analyze_sentiment)
    df['user_location'] = df['user'].apply(lambda user: user['location'])


    display(df[['creation_date', 'translated_text', 'sentiment','user_location']])

    daily_sentiments = df.groupby([df['creation_date'].dt.date, 'sentiment']).size().unstack(fill_value=0).reset_index()
    melted_df = daily_sentiments.melt(id_vars='creation_date', var_name='sentiment', value_name='count')
    fig = px.line(melted_df, x='creation_date', y='count', color='sentiment',
                  title=f'Sentiment Distribution for Hashtag {hashtag} Over Time',
                  labels={'creation_date': 'Date', 'count': 'Number of Tweets'},
                  line_shape='linear',
                  color_discrete_map={'Positive': 'green', 'Neutral': 'gray', 'Negative': 'red'})
    fig.update_layout(
        legend=dict(title='Sentiment', orientation='h', y=1.1, x=0.5),
        xaxis=dict(tickformat='%Y-%m-%d'),
        yaxis=dict(title='Number of Tweets', tickmode='linear'),
        autosize=True,
        template='plotly_dark'
    )
    fig.show()
else:
    print("No tweets found for the hashtag:", hashtag)


,creation_date,translated_text,sentiment,user_location
0,2024-04-10 12:47:22+00:00,Cricketer Hardik Pandya and Krunal Pandya with...,Neutral,Vrindavan
1,2024-04-11 09:59:13+00:00,Scenes before MI vs RCB.\n\n#MIvsRCB | #MIvRCB...,Neutral,India
2,2024-04-18 03:09:41+00:00,GOAT Rohit Sharma plays cricket today🐐\n\nBe r...,Positive,India
3,2024-04-15 11:03:48+00:00,When Virat Kohli played last time against SRH ...,Positive,heaven on earth
4,2024-04-12 05:48:06+00:00,Recpact to all Indian cricketers 🏏 \n#HardikP...,Neutral,RAM RAJYA
5,2024-04-13 16:58:00+00:00,Ishan about Hardik Pandya after the win agains...,Positive,
6,2024-04-18 08:00:06+00:00,Today PBKS vs MI\n#IPL2024 #RohitSharma \n#PBK...,Neutral,india
7,2024-04-15 01:03:07+00:00,#ChennaiSuperKings beat Mumbai Indians by 20 r...,Neutral,pathankot
8,2024-04-10 06:14:23+00:00,Sundarkand 🛕\n\nPANDYA'S HOME\n\n#Hardikpandya...,Neutral,
9,2024-04-12 05:39:09+00:00,#HardikPandya\nFinisher not yet finished 👀\nMo...,Positive,
